In [1]:
import json

In [2]:
def get_exp(exp):
    root_dir = f'sm_local/outputs_{exp}'
    loggings = f"{root_dir}/loggings.json"
    with open(loggings, 'r') as f:
        loggings = json.load(f)

    extraction = loggings['extraction']
    avg_ext_valid = sum([_['valid_ratio'] * _['num_questions_used'] for _ in extraction]) / sum([_['num_questions_used'] for _ in extraction])
    avg_ext_num_persona = sum([_['total_num_personas_extracted'] for _ in extraction]) / len(extraction)
    avg_ext_time = sum([_['extraction_time'] for _ in extraction]) / len(extraction)
    # print('num_personas:', avg_ext_num_persona)
    # print('valid_ratio:', avg_ext_valid)
    # print('time:', avg_ext_time)
    # print()

    clustering = loggings['clustering']
    avg_cluster_time = sum([_['clustering_time'] for _ in clustering]) / len(clustering)
    # print('time:', avg_cluster_time)
    # print()

    summarizing = loggings['summarizing']
    avg_sum_valid = sum([_['valid_ratio'] for _ in summarizing]) / len(summarizing)
    avg_sum_time = sum([_['summarizing_time'] for _ in summarizing]) / len(summarizing)
    # print('valid_ratio:', avg_sum_valid)
    # print('time:', avg_sum_time)
    # print()

    cleaning = loggings['cleaning']
    avg_clean_valid = sum([_['is_successful'] for _ in cleaning]) / len(cleaning)
    avg_clean_num_persona = sum([_['num_final_personas'] for _ in cleaning]) / len(cleaning)
    avg_clean_time = sum([_['cleaning_time'] for _ in cleaning]) / len(cleaning)
    # print('num_personas:', avg_clean_num_persona)
    # print('valid_ratio:', avg_clean_valid)
    # print('time:', avg_clean_time)

    res = [
        avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_time, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time
    ]

    return res

In [4]:
for llm in ['haiku', 'sonnet']:
    kmeans = 20
    survey = 'single'
    extraction_type = 'example'
    exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_time, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_time:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")

    kmeans = 10
    survey = 'single'
    extraction_type = 'example'
    exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_time, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_time:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")
    
    kmeans = 10
    survey = 'single'
    extraction_type = 'description'
    exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_time, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_time:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")


haiku;kmeans;20;single;example;1.000;752.800;436.514;85.202;0.993;44.802;0.600;21.600;69.847
haiku;kmeans;single;example;0.998;759.600;433.564;86.788;0.967;36.395;0.867;31.000;40.443
haiku;kmeans;single;description;0.716;225.200;225.480;30.138;0.987;18.987;1.000;18.867;13.228
sonnet;kmeans;20;single;example;0.998;725.600;1142.642;71.220;1.000;84.603;1.000;25.800;70.975
sonnet;kmeans;single;example;0.998;727.600;1060.569;71.632;0.993;58.143;0.933;16.533;40.472


KeyError: 'clustering'